### IPMOF - InterPenetrating Metal Organic Frameworks

#### Initialize necessary information

1. Read FF_Parameters excel file to get force field parameters for atoms
2. Initialize force field selection, cut_off radius and grid size for energy map

#### Read structural information for MOF files in a given directory
1. Read MOF files in ".mol2" format from given directory and create a list
2. Create MOF objects for structure files
3. Initialize structural information for the MOFs

#### Read simulation input parameters
1. Read simulation parameters from input file

#### Calculate energy map
1. Determine packing amount of the MOF
2. Calculate packed coordinates for the base MOF
3. Calculate energy map

#### Start interpenetration
1. Energy map + mobile_mof

In [3]:
import os
import math

# Load 3rd party libraries
import yaml

# Load interpenetration python libraries
from ipmof.crystal import MOF
from ipmof.forcefield import read_ff_parameters
from ipmof.energymap import energy_map, get_mof_list, get_uniq_atom_list
from ipmof.interpenetration import run_interpenetration, check_extension, save_extension
# --------------------------------------------------------------------------------------------------
# Get directories for simulation parameters and directories files
input_dir = os.getcwd()
sim_par_path = os.path.join(input_dir, 'sim_par.yaml')
sim_dir_path = os.path.join(input_dir, 'sim_dir_linux.yaml')

# Read sim par yaml file
sim_par = yaml.load(open(sim_par_path, 'r'))
sim_dir = yaml.load(open(sim_dir_path, 'r'))

# Read excel file containing force field information
force_field = read_ff_parameters(sim_dir['excel_file_path'], sim_par['force_field'])

### File Input Options
- <b> Built-in read_mol2 </b>
<pre>
uc_size, uc_angle, atom_names, atom_coors = read_mol2(mol2_path)
mof = MOF()
mof.initialize(mof.mol2_path)
</pre> 
- <b> Ase pdb read (cif read gives error) </b>
<pre>
from ase.io import read
mof_atoms = read(mof_dir, format='pdb')
mof_atoms.get_positions()                # Coordinates
mof_atoms.get_chemical_symbols()         # Atom names
mof_atoms.get_cell()                     # Cell vectors? (check)
mof_obj.get_number_of_atoms()            # Num of atoms
mof_obj.get_volume()                     # Unit cell volume

</pre>
- <b> Open babel </b>
<pre>
babel -icif *.cif -opdb *.pdb
</pre>

In [4]:
# Create list of MOFs
mol2_list = get_mof_list(sim_dir['mol2_dir'], '.mol2')
print(mol2_list)

base_mof_index = 4
mobile_mof_index = 4

['QIGBIR.mol2', 'OFODAP.mol2', 'UNIGEE.mol2', 'ZIF90.mol2', 'SAHYIK.mol2', 'KINFAQ.mol2', 'NUVWIL.mol2']


In [5]:
# Read mol2 files and initialize MOF objects
mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[base_mof_index])
base_mof = MOF(mol2_path)
base_mof.force_field(force_field)
print('Base MOF selected as: ', base_mof.name)

mol2_path = os.path.join(sim_dir['mol2_dir'], mol2_list[mobile_mof_index])
mobile_mof = MOF(mol2_path)
mobile_mof.force_field(force_field)
print('Mobile MOF selected as: ', mobile_mof.name)

Base MOF selected as:  SAHYIK
Mobile MOF selected as:  SAHYIK


In [6]:
extended_structure = base_mof.extend_unit_cell(sim_par['cut_off'])

print('Base MOF unit cell: ', base_mof.uc_size)
print('Packing factor:', base_mof.packing_factor)
print('Num of coor :', len(base_mof.packed_coors)*len(base_mof.packed_coors[0]))

Base MOF unit cell:  [25.669, 25.669, 25.669]
Packing factor: [3, 3, 3]
Num of coor : 11448


In [7]:
atom_list = get_uniq_atom_list([mobile_mof])
print('Calculating emap for', base_mof.name, 'with atoms:', atom_list['atom'])
emap = energy_map(sim_par, base_mof, atom_list)

Calculating emap for SAHYIK with atoms: ['C', 'H', 'Zn', 'O']


In [8]:
# Run interpenetration
summary, new_structures = run_interpenetration(sim_par, base_mof, mobile_mof, emap, atom_list)

In [9]:
# Get minimum energy structure by sorting total structure energies
min_energy_structure = sorted(new_structures, key=lambda k: k['energy'])[0]

In [10]:
# Check for collision in the extended unitcell of new structure and energy map
#sim_par['ext_cut_off'] = 30
collision = check_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

print('Collision:', collision)

Collision: False


In [11]:
# Get structure information for the interpenetrating structure
ext_structure = save_extension(sim_par, base_mof, mobile_mof, emap, atom_list, min_energy_structure)

In [15]:
# Extend MOF coordinates and get atom names and coordinates of extended unit cells of MOF object
extended_structure = base_mof.extend_unit_cell(sim_par['ext_cut_off'])

# Create new MOF objects for base and mobile MOFs
ext_base_mof = MOF(extended_structure, file_format='dict')
ext_mobile_mof = MOF(ext_structure, file_format='dict')

# Join base and mobile structure layers
joined_mof = ext_base_mof.join(ext_mobile_mof)

# Export to xyz format
joined_mof.export(sim_dir['export_dir'])

In [14]:
extended_structure['name']

KeyError: 'name'